In [1]:
import os 
import time
import copy
from tqdm import tqdm

import numpy as np 
import matplotlib.pyplot as plt
import scipy
from scipy import sparse, linalg, fft
from scipy.linalg import expm, sinm, cosm
import scipy.integrate as integrate
from scipy.integrate import quad
import pandas as pd

from joblib import Parallel, delayed
from numba import jit, njit, prange

import functools

# Locals
from hamiltonian_generation import make_H3_sparse

############ Macros ###############
np.random.seed(0)

# Physical constants
K = 10 # number of fermionic modes
J = 4 # ~"energy scale"
Q_COUPLING = 3 # order of coupling, don't want to use the letter 'Q' because that denotes the supercharge
N = 2*K # number of fermions
N_DIM = 2**K # Hilbert space dimension

N_SAMPLES = 100 # number of samples to generate
N_JOBS = 20 # number of jobs to run in parallel


Just like normal SYK, we're still interested in Majorana fermions...

In [2]:
H3_func, psi, psi_pairs, psi_tris = make_H3_sparse(K, K, precompute_pairs=True, precompute_tris=True)

sigma_c = np.sqrt(2*J/(N**2))
C_test = np.random.normal(0, sigma_c, size=(N, N, N))
Q_test = H3_func(C_test).toarray()

iv = np.linalg.eigvalsh(Q_test)
plt.figure()
plt.hist(iv, bins=30)
plt.title(r"Eigenvalues of the supercharge, $Q$")
plt.show()
plt.close()

print("Q hermitian: ", linalg.ishermitian(Q_test))

Q hermitian:  True


Welp

## 1. Compute Hamiltonian

This one is straightforward

In [ ]:
H_test = Q_test@Q_test
print(linalg.ishermitian(H_test))

## 2. Check equation 1.4

$H = Q^2 = E_0 + \sum_{1\le i<j<k<l\le N} J_{ijkl} \psi^i \psi^j \psi^k \psi^l$

Where

$E_0 = \sum_{1\le i<j<k \le N} C_{ijk}^2$

and

$J_{ijkl} = -\frac{1}{8} \sum_a C_{a[ij}C_{kl]a}$

### 2.a. Let's start by computing the easy one

In [ ]:
def get_E0(C):
    E0 = 0
    for i in range(N-2):
        for j in range(i+1, N-1):
            for k in range(j+1, N):
                E0 += C[i,j,k]**2
    return E0

def get_E02(C):
    C2 = C**2
    #C2 = C2[:N-2, 1:N-1, 2:N]
    E0 = np.sum(np.sum(np.sum(C2[:N-2], axis=0)[1:N-1], axis=0)[2:N], axis=0)
    return E0

E0_test = get_E0(C_test)
E0_test2 = get_E02(C_test)
print(E0_test)
print(E0_test2)

In [ ]:
a = C_test[0].real

### 2.b. And the hard one...

First, we need a function to compute the levi-civita value of some numbers (i, j, k, l)

In [ ]:
# TODO: Not very elegant
def antisymm_4(i, j, k, l): # <-- Assumes i,j,k,l already ordered
    if (i==j) or (i==k) or (i==l) or (j==k) or (j==l) or (k==l):
        return 0

    out = {}
    
    # i first 

        # j second
    out[(i, j, k, l)] = 1
    out[(i, j, l, k)] = -1

        # k second
    out[(i, k, j, l)] = -1
    out[(i, k, l, j)] = 1

        # l second
    out[(i, l, k, j)] = -1
    out[(i, l, j, k)] = 1



    # j first

        # i second
    out[(j, i, k, l)] = -1
    out[(j, i, l, k)] = 1

        # k second
    out[(j, k, i, l)] = 1
    out[(j, k, l, i)] = -1

        # l second
    out[(j, l, k, i)] = 1
    out[(j, l, i, k)] = -1


    # k first

        # i second
    out[(k, i, j, l)] = 1
    out[(k, i, l, j)] = -1

        # j second
    out[(k, j, i, l)] = -1
    out[(k, j, l, i)] = 1

        # l second
    out[(k, l, i, j)] = 1
    out[(k, l, j, i)] = -1



    # l first
    
        # j second
    out[(l, i, j, k)] = -1
    out[(l, i, k, j)] = 1

        # j second
    out[(l, j, i, k)] = 1
    out[(l, j, k, i)] = -1

        # k second
    out[(l, k, j, i)] = 1
    out[(l, k, i, j)] = -1

    return out

out = antisymm_4(0,1,2,3)
print(sum(out.values()))

In [ ]:
def get_J_a(C, a, i, j, k, l):
    C_left = C[a,:,:]
    C_right = C[:,:,a]

    permutations = antisymm_4(i, j, k, l)
    out = 0
    for perm, sign in permutations.items():
        out += C_left[perm[:2]]*C_right[perm[2:]]*sign
    return out

def get_J_ijkl(C, i, j, k, l):
    sum = 0
    for a in range(N):
        sum += get_J_a(C, a, i, j, k, l)
    out = -sum/8
    return out


In [ ]:
H_long_way = E0_test2*np.identity(N_DIM)

tic = time.time()

for i in range(N-3):
    for j in range(i+1, N-2):
        for k in range(j+1, N-1):
            for l in range(k+1, N):
                H_long_way += get_J_ijkl(C_test, i, j, k, l)*psi_pairs[i*N+j]@psi_pairs[k*N+l]

duration = time.time() - tic
print(f"Duration: {duration//60} minutes, {duration%60} seconds")

In [ ]:
print(np.allclose(H_test, H_long_way))

In [ ]:
abs_diff = np.array(np.abs(H_test - H_long_way))
print(np.sum(np.sum(abs_diff)))
